Metabolome Microbiome Network  

# Impute Reactome Species with HMDB Information

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
tree = ET.parse('hmdb_metabolites.xml')
root = tree.getroot()

In [4]:
# This parsing step will take quite some time (few minutes) as the hmdb_metabolites.xml files is pretty big
hmdb_synonyms = {}
hmdb_mets = [['name', 'hmdb_id', 'iupac', 'kegg_id', 'foodb_id', 'chemspider_id', 'drugbank_id', 'pdb_id',
             'chebi_id', 'pubchem_compound_id', 'wikipedia_id', 'bigg_id', 'vmh_id']]

for child in root:
    synonym_set = set()
    
    for elem in child:
        if elem.tag == r'{http://www.hmdb.ca}name':
            name = elem.text
            synonym_set.add(name)

        if elem.tag == r'{http://www.hmdb.ca}synonyms':
            for synonym in elem:
                if synonym.text != name:
                    synonym_set.add(synonym.text)
                    synonym_set.add(synonym.text.lower())

        if elem.tag == r'{http://www.hmdb.ca}accession':
            hmdb_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}iupac_name':
            iupac_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}kegg_id':
            kegg_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}foodb_id':
            foodb_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}chemspider_id':
            chemspider_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}drugbank_id':
            drugbank_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}pdb_id':
            pdb_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}chebi_id':
            chebi_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}pubchem_compound_id':
            pubchem_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}wikipedia_id':
            wikipedia_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}bigg_id':
            bigg_id = elem.text

        if elem.tag == r'{http://www.hmdb.ca}vmh_id':
            vmh_id = elem.text
            
    hmdb_synonyms.update({hmdb_id: synonym_set})
    
    metabolite = [name, hmdb_id, iupac_id, kegg_id, foodb_id, chemspider_id, drugbank_id, pdb_id, chebi_id, pubchem_id, wikipedia_id, bigg_id, vmh_id]
    hmdb_mets.append(metabolite)

In [5]:
df_hmbd = pd.DataFrame(hmdb_mets[1:], columns=hmdb_mets[0])

In [11]:
species_reactome = pd.read_csv('extracted_reactome_data/reactome_species.csv', dtype=str)

In [12]:
species_reactome

,species_id,species_name,species_reactome_id,entity_type,CHEBI,uniprot,ensembl,GRAC,pubchem,ncbi,mirbase,ENA,KEGG
0,species_10637177,"PEX2:PEX10:PEX12:PEX5S,L:Ub:UBE2D1,2,3:PEX13:P...",R-CEL-8953942,Reactome Complex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,P35129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,Q9U1U4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,Q20617,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,species_10635551,RNF152:RRAGA:GDP:Ub:UBE2N,R-CEL-8938812,Reactome Complex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238513,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,E7FBD9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238514,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,A0A2R8QFW3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238515,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,F1RBL0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238516,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,E9QH31,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Check for every metabolite in HMDB if the name matches any of the known synonyms and if so, return the hmdb_id
def find_hmdb_id(name):
    for synonym, synonym_set in hmdb_synonyms.items():
        if name in synonym_set:
            return synonym

In [14]:
# This step might take a very long time
# (the efficiency of the search could definitely be improved)
species_reactome['hmdb_id'] = species_reactome.species_name.apply(find_hmdb_id)

In [15]:
species_reactome[species_reactome.hmdb_id.notna()]

,species_id,species_name,species_reactome_id,entity_type,CHEBI,uniprot,ensembl,GRAC,pubchem,ncbi,mirbase,ENA,KEGG,hmdb_id
13,species_113582,ADP,R-ALL-113582,Reactome SimpleEntity,456216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0001341
15,species_426925,PC,R-ALL-426925,Reactome SimpleEntity,16110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0008102
17,species_76577,AMP,R-ALL-76577,Reactome SimpleEntity,16027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0000045
23,species_113592,ATP,R-ALL-113592,Reactome SimpleEntity,30616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0000538
25,species_74016,Ca2+,R-ALL-74016,Reactome SimpleEntity,29108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0000464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235046,species_74819,IDA,R-HSA-74819,Reactome GenomeEncodedEntity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0011753
236749,species_10610155,trp-2,R-CEL-2089937,Reactome EntityWithAccessionedSequence,NaN,Q8I6Y9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0033188
236896,species_9996018,Elastin,R-CFA-2161232,Reactome Polymer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0041419
238292,species_10259431,Elastin,R-DRE-2161232,Reactome Polymer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HMDB0041419


In [16]:
# Merge Reactome and HMDB metabolites
reactome_hmdb_merge = pd.merge(species_reactome, df_hmbd, on='hmdb_id', how='left')

In [17]:
reactome_hmdb_merge[reactome_hmdb_merge.name.notna()]

,species_id,species_name,species_reactome_id,entity_type,CHEBI,uniprot,ensembl,GRAC,pubchem,ncbi,...,kegg_id,foodb_id,chemspider_id,drugbank_id,pdb_id,chebi_id,pubchem_compound_id,wikipedia_id,bigg_id,vmh_id
13,species_113582,ADP,R-ALL-113582,Reactome SimpleEntity,456216,NaN,NaN,NaN,NaN,NaN,...,C00008,FDB021817,5800,None,None,16761,6022,Adenosine_diphosphate,33496,ADP
15,species_426925,PC,R-ALL-426925,Reactome SimpleEntity,16110,NaN,NaN,NaN,NaN,NaN,...,None,FDB025293,None,None,None,76073,24778936,None,None,None
17,species_76577,AMP,R-ALL-76577,Reactome SimpleEntity,16027,NaN,NaN,NaN,NaN,NaN,...,C00020,FDB030677,5858,DB00131,None,16027,6083,Adenylic_acid,33534,AMP
23,species_113592,ATP,R-ALL-113592,Reactome SimpleEntity,30616,NaN,NaN,NaN,NaN,NaN,...,C00002,FDB030683,5742,DB00171,None,15422,5957,Adenosine_triphosphate,33477,ATP
25,species_74016,Ca2+,R-ALL-74016,Reactome SimpleEntity,29108,NaN,NaN,NaN,NaN,NaN,...,C00076,FDB003513,266,None,None,29108,271,Calcium,33764,CA2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235046,species_74819,IDA,R-HSA-74819,Reactome GenomeEncodedEntity,NaN,NaN,NaN,NaN,NaN,NaN,...,C19911,FDB028424,8557,None,None,24786,8897,Iminodiacetic acid,None,None
236749,species_10610155,trp-2,R-CEL-2089937,Reactome EntityWithAccessionedSequence,NaN,Q8I6Y9,NaN,NaN,NaN,NaN,...,C14416,FDB011200,4447540,None,None,None,5284476,None,None,None
236896,species_9996018,Elastin,R-CFA-2161232,Reactome Polymer,NaN,NaN,NaN,NaN,NaN,NaN,...,C00373,FDB021364,None,None,None,None,85083221,None,None,None
238292,species_10259431,Elastin,R-DRE-2161232,Reactome Polymer,NaN,NaN,NaN,NaN,NaN,NaN,...,C00373,FDB021364,None,None,None,None,85083221,None,None,None


In [18]:
reactome_hmdb_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238518 entries, 0 to 238517
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   species_id           238518 non-null  object
 1   species_name         238515 non-null  object
 2   species_reactome_id  238518 non-null  object
 3   entity_type          238518 non-null  object
 4   CHEBI                8184 non-null    object
 5   uniprot              166323 non-null  object
 6   ensembl              1310 non-null    object
 7   GRAC                 579 non-null     object
 8   pubchem              30 non-null      object
 9   ncbi                 28 non-null      object
 10  mirbase              52 non-null      object
 11  ENA                  42 non-null      object
 12  KEGG                 3 non-null       object
 13  hmdb_id              991 non-null     object
 14  name                 991 non-null     object
 15  iupac                985 non-null 

In [19]:
# As some identifiers occur in Reactome as well as in HMDB (e.g. CHEBI and chebi_id) it is necessary to merge the information
reactome_hmdb_merge['CHEBI'] = reactome_hmdb_merge.apply(lambda row: row.CHEBI if pd.notna(row.CHEBI) else (row.chebi_id if pd.notna(row.chebi_id) else float('nan')), axis=1)
reactome_hmdb_merge['KEGG'] = reactome_hmdb_merge.apply(lambda row: row.KEGG if pd.notna(row.KEGG) else (row.kegg_id if pd.notna(row.kegg_id) else float('nan')), axis=1)
reactome_hmdb_merge['pubchem'] = reactome_hmdb_merge.apply(lambda row: row.pubchem if pd.notna(row.pubchem) else (row.pubchem_compound_id if pd.notna(row.pubchem_compound_id) else float('nan')), axis=1)

In [20]:
reactome_hmdb_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238518 entries, 0 to 238517
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   species_id           238518 non-null  object
 1   species_name         238515 non-null  object
 2   species_reactome_id  238518 non-null  object
 3   entity_type          238518 non-null  object
 4   CHEBI                8370 non-null    object
 5   uniprot              166323 non-null  object
 6   ensembl              1310 non-null    object
 7   GRAC                 579 non-null     object
 8   pubchem              1003 non-null    object
 9   ncbi                 28 non-null      object
 10  mirbase              52 non-null      object
 11  ENA                  42 non-null      object
 12  KEGG                 834 non-null     object
 13  hmdb_id              991 non-null     object
 14  name                 991 non-null     object
 15  iupac                985 non-null 

In [21]:
# We drop the columns we don't need anymore
reactome_hmdb_merge_cleaned = reactome_hmdb_merge.drop(columns=['chebi_id', 'kegg_id', 'pubchem_compound_id'])

In [32]:
reactome_hmdb_merge_cleaned

,species_id,species_name,species_reactome_id,entity_type,CHEBI,uniprot,ensembl,GRAC,pubchem,ncbi,...,hmdb_id,name,iupac,foodb_id,chemspider_id,drugbank_id,pdb_id,wikipedia_id,bigg_id,vmh_id
0,species_10637177,"PEX2:PEX10:PEX12:PEX5S,L:Ub:UBE2D1,2,3:PEX13:P...",R-CEL-8953942,Reactome Complex,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,P35129,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,Q9U1U4,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,species_10603252,"UBE2D1,2,3",R-CEL-1234120,Reactome DefinedSet,NaN,Q20617,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,species_10635551,RNF152:RRAGA:GDP:Ub:UBE2N,R-CEL-8938812,Reactome Complex,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238513,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,E7FBD9,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
238514,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,A0A2R8QFW3,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
238515,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,F1RBL0,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
238516,species_10318510,Class I Peroxisomal Membrane Proteins,R-DRE-9603783,Reactome DefinedSet,NaN,E9QH31,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


&rarr; As we can see we have been able to match 991 metabolites to HMDB